# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [8]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [7]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

### Question 1: What is the sum of the outputs of the generator for limit = 5?

In [3]:

def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)
sum = 0

for sqrt_value in generator:
    sum+= sqrt_value

print("sum= ", sum)

sum=  8.382332347441762


### Question 2: What is the 13th number yielded

In [6]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)
s = 1

for sqrt_value in generator:
    if s ==13:
      print(s,"- ",sqrt_value)
    s+=1

13 -  3.605551275463989


**Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.**


In [9]:
import dlt

# define the connection to load to.
# use duckdb
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

# load the first generator to a table at the pipeline destnation
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")
# the outcome metadata
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.58 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708004036.5578618 is LOADED and contains no failed jobs


**Append the second generator to the same table as the first.**

In [10]:
# load the seconed generator to a table at the pipeline destnation
info2 = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")
# the outcome metadata
print(info2)

Pipeline dlt_colab_kernel_launcher load step completed in 0.36 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708004200.9944086 is LOADED and contains no failed jobs


### Question 3: After correctly appending the data, calculate the sum of all ages of people.

In [12]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

P = conn.sql("SELECT * FROM people").df()
display(P)

s = conn.sql("SELECT sum(age) FROM people").df()
display(s)



Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708004036.5578618,+6y6UjnnBO82OQ,None
1,2,Person_2,27,City_A,1708004036.5578618,1ZDPgwC5puwJcA,None
2,3,Person_3,28,City_A,1708004036.5578618,2UPEnM3mFkErqA,None
3,4,Person_4,29,City_A,1708004036.5578618,JcDQOFZJyrOu5Q,None
4,5,Person_5,30,City_A,1708004036.5578618,60oSISmeAO7LiQ,None
5,3,Person_3,33,City_B,1708004200.9944086,L2bNSO2BOa09Vg,Job_3
6,4,Person_4,34,City_B,1708004200.9944086,Y0+njsOt0BvC0A,Job_4
7,5,Person_5,35,City_B,1708004200.9944086,laiyG9eyAUly0g,Job_5
8,6,Person_6,36,City_B,1708004200.9944086,+wOOZDMZbGqDZw,Job_6
9,7,Person_7,37,City_B,1708004200.9944086,fvXUW8Mjhajlxw,Job_7


,sum(age)
0,353.0


**Merge a generator**
Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

In [13]:
# define the connection to load to.
# duckdb
merge_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators_merge')

# load the first generator to a table at the pipeline destnation
info = merge_pipeline.run(people_1(),
										table_name="people_merge",
										write_disposition="replace",
                    primary_key='ID')
# the outcome metadata
print(info)


Pipeline dlt_colab_kernel_launcher load step completed in 0.62 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators_merge
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708005155.1084175 is LOADED and contains no failed jobs


**Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.**

In [14]:
# load the seconed generator to a table at the pipeline destnation
info2 = merge_pipeline.run(people_2(),
										table_name="people_merge",
										write_disposition="merge",
                    primary_key='ID')
# the outcome metadata
print(info2)

Pipeline dlt_colab_kernel_launcher load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators_merge
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708005163.8247335 is LOADED and contains no failed jobs



**After loading, you should have a total of 8 records, and ID 3 should have age 33.**

### Question 4: Calculate the sum of ages of all the people loaded as described above.

In [15]:
# show outcome

import duckdb

conn = duckdb.connect(f"{merge_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{merge_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_merge table below:")

P = conn.sql("SELECT * FROM people_merge").df()
display(P)

s = conn.sql("SELECT sum(age) FROM people_merge").df()
display(s)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_merge        │
└─────────────────────┘




 people_merge table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708005155.1084175,9IqrxR+Emz9eCQ,None
1,2,Person_2,27,City_A,1708005155.1084175,vCD1EH1Js+3LmQ,None
2,8,Person_8,38,City_B,1708005163.8247335,8syfveUWnvpEXg,Job_8
3,7,Person_7,37,City_B,1708005163.8247335,6g9FiDI/GGP6Yg,Job_7
4,5,Person_5,35,City_B,1708005163.8247335,8WeDX6/7Eh7dSQ,Job_5
5,4,Person_4,34,City_B,1708005163.8247335,/xs948Q0jlQXrQ,Job_4
6,3,Person_3,33,City_B,1708005163.8247335,flpAZo4In9hVKw,Job_3
7,6,Person_6,36,City_B,1708005163.8247335,gOWR2HtYL30JNA,Job_6


,sum(age)
0,266.0
